In [59]:
# Initial imports
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [60]:
# Load the crypto_data.csv dataset.
file_path = "Resources/NYC_BICYCLE_NETWORK.csv"
bike_lane_df = pd.read_csv(file_path)
print(bike_lane_df.shape)
bike_lane_df.head(5)

(20824, 17)


,the_geom,segmentid,Shape_Le_1,boro,street,fromstreet,tostreet,onoffst,facilitycl,allclasses,bikedir,lanecount,ft_facilit,tf_facilit,ft2facilit,tf2facilit,comments
0,MULTILINESTRING ((-74.24992754986717 40.500876...,53,131.771990,5,CONFERENCE HOUSE PARK GREENWAY,HYLAN BLVD,SWINNERTON ST,OFF,I,I,2,2,Greenway,Greenway,NaN,NaN,NaN
1,MULTILINESTRING ((-74.24915648792326 40.498717...,57,268.689582,5,CONFERENCE HOUSE PARK GREENWAY,HYLAN BLVD,SWINNERTON ST,OFF,I,I,2,2,Greenway,Greenway,NaN,NaN,NaN
2,MULTILINESTRING ((-74.24934702119957 40.499440...,59,420.193253,5,CONFERENCE HOUSE PARK GREENWAY,HYLAN BLVD,SWINNERTON ST,OFF,I,I,2,2,Greenway,Greenway,NaN,NaN,NaN
3,MULTILINESTRING ((-74.25185153133741 40.502805...,61,238.242198,5,CONFERENCE HOUSE PARK GREENWAY,HYLAN BLVD,SWINNERTON ST,OFF,I,I,2,2,Greenway,Greenway,NaN,NaN,NaN
4,MULTILINESTRING ((-74.2513327304861 40.5022891...,64,646.293832,5,CONFERENCE HOUSE PARK GREENWAY,HYLAN BLVD,SWINNERTON ST,OFF,I,I,2,2,Greenway,Greenway,NaN,NaN,NaN


In [61]:
bike_lane_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20824 entries, 0 to 20823
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   the_geom    20824 non-null  object 
 1   segmentid   20824 non-null  int64  
 2   Shape_Le_1  20824 non-null  float64
 3   boro        20824 non-null  int64  
 4   street      20824 non-null  object 
 5   fromstreet  20824 non-null  object 
 6   tostreet    20824 non-null  object 
 7   onoffst     20824 non-null  object 
 8   facilitycl  20824 non-null  object 
 9   allclasses  20824 non-null  object 
 10  bikedir     20824 non-null  object 
 11  lanecount   20824 non-null  int64  
 12  ft_facilit  15430 non-null  object 
 13  tf_facilit  15264 non-null  object 
 14  ft2facilit  15 non-null     object 
 15  tf2facilit  48 non-null     object 
 16  comments    948 non-null    object 
dtypes: float64(1), int64(3), object(13)
memory usage: 2.7+ MB


In [62]:
#bike_lane_df.dropna(axis=0, how='all')
#print(bike_lane_df.shape)
bike_lane_df1=bike_lane_df.drop(["ft2facilit","tf2facilit", "comments"], axis=1)
bike_lane_df2=bike_lane_df1.dropna(axis=0, how='all')
print(bike_lane_df2.shape)

(20824, 14)


In [63]:
# get the unique street name form the dataframe as array and count the number
len(bike_lane_df2["street"].unique())

1292

In [64]:
bike_lane_df2.drop_duplicates(subset = ["street"])

,the_geom,segmentid,Shape_Le_1,boro,street,fromstreet,tostreet,onoffst,facilitycl,allclasses,bikedir,lanecount,ft_facilit,tf_facilit
0,MULTILINESTRING ((-74.24992754986717 40.500876...,53,131.771990,5,CONFERENCE HOUSE PARK GREENWAY,HYLAN BLVD,SWINNERTON ST,OFF,I,I,2,2,Greenway,Greenway
10,MULTILINESTRING ((-74.22765169907424 40.508199...,735,690.286899,5,HYLAN BLVD,PAGE AV,POILLON AV,ON,II,II,2,2,Standard,Standard
13,MULTILINESTRING ((-74.22124840590146 40.505101...,841,836.449298,5,MOUNT LORETTO GREENWAY,HYLAN BLVD,HYLAN BLVD,OFF,I,I,2,2,Greenway,Greenway
28,MULTILINESTRING ((-74.19697470899112 40.557563...,2416,440.773031,1,79 ST NORTH ACCESS RAMP,W 79 ST,HUDSON RIVER GREENWAY 79TH ST LINK,OFF,II,"II,III",2,2,Curbside,Sharrows
45,MULTILINESTRING ((-74.16694849149822 40.602796...,4871,271.541232,5,ARLENE ST,LAMBERTS LA,SIGNS RD,ON,II,II,2,2,Standard,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20754,MULTILINESTRING ((-73.84227808018568 40.741245...,177286,63.135302,4,AMPHITHEATRE BR,EDERLE PROMENADE,MERIDIAN RD,OFF,I,I,2,2,Greenway,Greenway
20782,MULTILINESTRING ((-74.02916023126342 40.640774...,17575,266.682165,3,WAKEMAN PL,COLONIAL RD,RIDGE BL,OFF,I,I,2,2,Sidewalk,Sidewalk
20786,MULTILINESTRING ((-73.79512969395029 40.866381...,321550,422.638126,2,ORCHARD BEACH BOARDWALK LINK,PARK DRIVE GREENWAY,PRCHARD BEACH BOARDWALK,OFF,L,L,2,0,NaN,NaN
20802,MULTILINESTRING ((-73.98875574891973 40.695018...,162715,39.491486,3,JOHNSON ST,ADAMS ST,JAY ST,ON,I,"III,I",2,2,Sharrows,Protected Path


In [65]:
bike_lane_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20824 entries, 0 to 20823
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   the_geom    20824 non-null  object 
 1   segmentid   20824 non-null  int64  
 2   Shape_Le_1  20824 non-null  float64
 3   boro        20824 non-null  int64  
 4   street      20824 non-null  object 
 5   fromstreet  20824 non-null  object 
 6   tostreet    20824 non-null  object 
 7   onoffst     20824 non-null  object 
 8   facilitycl  20824 non-null  object 
 9   allclasses  20824 non-null  object 
 10  bikedir     20824 non-null  object 
 11  lanecount   20824 non-null  int64  
 12  ft_facilit  15430 non-null  object 
 13  tf_facilit  15264 non-null  object 
dtypes: float64(1), int64(3), object(10)
memory usage: 2.4+ MB


In [66]:
bike_lane_df2.to_csv("NYC_BIKE_LANE.csv")

In [68]:
bike_lane_df2["street"].unique()

array(['CONFERENCE HOUSE PARK GREENWAY', 'HYLAN BLVD',
       'MOUNT LORETTO GREENWAY', ..., 'ORCHARD BEACH BOARDWALK LINK',
       'JOHNSON ST', 'AVE H PED UNDERPASS'], dtype=object)